In [ ]:
import torch
import torch.nn as nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import models
from torchvision import transforms
from torchvision.datasets import ImageFolder

c:\Users\knudc\anaconda3\envs\NLP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 하이퍼 파라미터 설정
hyperparams = {"batch_size":4, 
               "learning_rate":0.0001,
               "epochs":5,
               "transform":transforms.Compose([transforms.Resize(256),      # 256으로 크기 설정한 이후 기준값인 224로 설정하는 순서
                                               transforms.CenterCrop(224),  # 물체를 더 잘 식별하기 위해 가장자리 제거 목적으로 CenterCrop
                                               transforms.ToTensor(),       # 텐서화
                                               transforms.Normalize(mean=[0.48235,0.45882,0.40784],  # 평균 : 이미지넷 데이터세트에 대한 대푯값
                                                                    std=[0.229,0.224,0.225])])}      # 표준편차 : RGB 값을 0.0~1.0 사잇값으로 정규화한 값

In [4]:
# 데이터셋 생성하기
train_dataset = ImageFolder('../data/image/pet/train', transform=hyperparams['transform'])  # 딕셔너리 key값을 통해 정보 불러오기
test_dataset = ImageFolder('../data/image/pet/test', transform=hyperparams['transform'])

In [5]:
# 데이터로더 생성하기
# shuffle = True : 일반화 성능 향상, 과적합 방지
# drop_last = True : 마지막 배치 제거 - 마지막 배치가 지정한 batch_size보다 작을 수 있음 이를 방지
train_dataloader = DataLoader(train_dataset, batch_size=hyperparams['batch_size'], shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=hyperparams['batch_size'], shuffle=True, drop_last=True)

In [6]:
# 사전 학습된 VGG16 모델 불러오기
model = models.vgg16(weights='VGG16_Weights.IMAGENET1K_V1')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\knudc/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:17<00:00, 31.9MB/s] 


In [7]:
# 모델 층별 추출 
FeatureLayers = model.features                # 이미지 특징 추출 부분 (Feature Map 추출)
ClassifierLayers = model.classifier           # 분류기 부분 (Full-Connected)

In [8]:
# 각 층 확인하기 : 미세조정을 하기 위해 정보 확인
print(FeatureLayers)
print(ClassifierLayers)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [9]:
# VGG16 모델 미세 조정
model.classifier[6] = nn.Linear(4096, len(train_dataset.classes))  

In [ ]:
# 학습 관련 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)    # to(device) : 모델과 데이터를 지정한 장치로 옮기는 작업 수행 
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=hyperparams['learning_rate'])

In [11]:
# 학습 
for epoch in range(hyperparams['epochs']):
    cost = 0.0
    
    for images,classes in train_dataloader:
        images = images.to(device)
        classes = classes.to(device)
        
        output = model(images)
        loss = criterion(output,classes)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        cost += loss
        
    cost = cost / len(train_dataloader)
    print(f'Epoch : {epoch+1:4d}, Cost : {cost:.3f}')

Epoch :    1, Cost : 0.117


KeyboardInterrupt: 

In [ ]:
# 검증 
with torch.no_grad():
    model.eval()
    
    accuracy = 0.0
    for images, classes in test_dataloader:
        images = images.to(device)
        classes = classes.to(device)
        
        outputs = model(images)
        probs = F.softmax(outputs, dim=-1)
        outputs_classes = torch.argmax(probs, dim=-1)
        
        accuracy += int(torch.eq(classes, outputs_classes).sum())
        print(f'acc@1 : {accuracy / (len(test_dataloader)*hyperparams['batch_size'])*100:.2f}%')
        
torch.save(model.state_dict(), '../models/VGG16.pt')
print( "Saved the model weights" )

In [ ]:
# 예측 